# LangChain to prompt programmatically in Python

1. Create an account on [OpenAI Platform](https://platform.openai.com/)
2. Create a project
3. Create an API Key in the project (Settings > Project > API Keys)
4. Paste it into the `.env` file under the variable `OPENAI_API_KEY`

## Load model

In [9]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff62502150>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff63651ca0>, root_client=<openai.OpenAI object at 0xffff84bf9220>, root_async_client=<openai.AsyncOpenAI object at 0xffff62501730>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompting seralization

### Load markdown template

In [10]:
with open('../prompts/ES.md', 'r') as file:
    template = file.read()
    
template

'La siguiente tabla muestra el retorno del activo financiero con ticker {TICKER}, calculado como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. Esta comparación se realiza respetando la frecuencia temporal constante observada en la serie (por ejemplo, diaria, semanal o mensual).\n\nTABLA:\n\n{TABLA}\n\nAnaliza los valores extremos, tanto positivos como negativos, y proporciona una explicación fundamentada para cada uno. Incluye, en la medida de lo posible, noticias relacionadas que puedan justificar dichas variaciones, junto con su fecha de publicación y un enlace a la fuente.'

### Create prompt object

In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    template_format='f-string',
)

prompt

PromptTemplate(input_variables=['TABLA', 'TICKER'], input_types={}, partial_variables={}, template='La siguiente tabla muestra el retorno del activo financiero con ticker {TICKER}, calculado como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. Esta comparación se realiza respetando la frecuencia temporal constante observada en la serie (por ejemplo, diaria, semanal o mensual).\n\nTABLA:\n\n{TABLA}\n\nAnaliza los valores extremos, tanto positivos como negativos, y proporciona una explicación fundamentada para cada uno. Incluye, en la medida de lo posible, noticias relacionadas que puedan justificar dichas variaciones, junto con su fecha de publicación y un enlace a la fuente.')

### Save prompt to JSON

In [12]:
path = '../prompts/ES.json'
prompt.save(path)

## Chaining with LLMs

### Define chain

In [13]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['TABLA', 'TICKER'], input_types={}, partial_variables={}, template='La siguiente tabla muestra el retorno del activo financiero con ticker {TICKER}, calculado como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. Esta comparación se realiza respetando la frecuencia temporal constante observada en la serie (por ejemplo, diaria, semanal o mensual).\n\nTABLA:\n\n{TABLA}\n\nAnaliza los valores extremos, tanto positivos como negativos, y proporciona una explicación fundamentada para cada uno. Incluye, en la medida de lo posible, noticias relacionadas que puedan justificar dichas variaciones, junto con su fecha de publicación y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff62502150>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff63651ca0>, root_client=<openai.Ope

### Preprocess input data

In [14]:
ticker = 'AAPL'
freq = 'D'

In [15]:
import utils

df = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=5
)

df

[*********************100%***********************]  1 of 1 completed


Ticker,AAPL
Date,
2024-01-03,-0.748758
2024-01-04,-1.270005
...,...
2024-12-27,-1.324213
2024-12-30,-1.326343


### Invoke chain with input data

In [16]:
output = chain.invoke({'TABLA': df, 'TICKER': ticker})
print(output)

A lo largo de 2024, las acciones de Apple (AAPL) experimentaron fluctuaciones significativas, tanto al alza como a la baja. A continuación, se analizan algunos de los movimientos más extremos y las posibles causas detrás de ellos:

**1. Caída significativa a principios de enero de 2024:**

En los primeros días de enero, las acciones de Apple sufrieron una notable disminución. El 2 de enero, Barclays rebajó la calificación de Apple debido a ventas mediocres del iPhone y falta de crecimiento en otros productos. Esta noticia provocó una caída de aproximadamente el 3% en el precio de las acciones, reduciendo la capitalización de mercado de Apple en cerca de 90,000 millones de dólares. ([es.investing.com](https://es.investing.com/news/stock-market-news/apple-pierde-los-3-billones-por-que-caen-acciones-en-el-primer-dia-del-2024-2528595?utm_source=openai))

**2. Nueva rebaja de calificación y caída adicional en enero de 2024:**

El 4 de enero, Piper Sandler también rebajó la calificación de A

## Export output to file

In [17]:
with open(f'reports/{ticker}_search.md', 'w') as file:
    file.write(output)